<a href="https://colab.research.google.com/github/matchbou/Public/blob/master/conversation_ja-01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


--- 
```
Copyright 2019 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.
```
---

## Google Cloud API の認証情報を入力

Google Cloud API を REST インタフェースから利用するために、 [API Key](https://console.cloud.google.com/apis/credentials) を利用します。 Google Cloud Console から API Key をコピーしましょう。



In [1]:
import getpass
APIKEY = getpass.getpass()

··········


## Cloud Speech-to-Text API を使ってみよう !

**Google Cloud Speech-to-Text** では、使いやすい API で高度なニューラル ネットワーク モデルを適用し、音声をテキストに変換できます。

API Discovery Service を利用して **Cloud Speech-to-Text API** を発見します。 Cloud Speech-to-Text の REST API 仕様は [こちら](https://cloud.google.com/speech-to-text/docs/reference/rest/) に解説されています。



In [0]:
from googleapiclient.discovery import build

speech_service = build('speech', 'v1p1beta1', developerKey=APIKEY)

### 音声データの準備

音声録音のための関数 ```record_audio``` を定義しましょう。

In [0]:
#@title このセルを実行して record_audio を定義

# Install required libraries and packages
!pip install -qq pydub
!apt-get -qq update
!apt-get -qq install -y ffmpeg

# Define record_audio
import base64
import google.colab
import pydub

from io import BytesIO

def record_audio(file_id, framerate=16000, file_format='flac'):
  # Record webm file from Colaboratory.
  audio = google.colab._message.blocking_request(
    'user_media',
    {
      'audio': True,
      'video': False,
      'duration': -1
    },
    timeout_sec=600)

  # Convert web file into in_memory file.
  mfile = BytesIO(base64.b64decode(audio[audio.index(',')+1:]))

  # Store webm file locally.
  with open('{0}.webm'.format(file_id), 'wb') as f:
    mfile.seek(0)
    f.write(mfile.read())

  # Open stored web file and save it as wav with sample_rate=16000
  output_file = '{0}.{1}'.format(file_id, file_format)
  _ = pydub.AudioSegment.from_file('{0}.webm'.format(file_id), codec='opus')
  _.set_frame_rate(16000).export(output_file, format=file_format)

  return output_file

```record_audio``` を実行して音声を録音しましょう。

In [6]:
record_audio('ja-sample')

'ja-sample.flac'

録音結果を確認しましょう。

In [7]:
from IPython.display import Audio

Audio('ja-sample.flac', rate=16000)

### 音声認識の実行

Cloud Speech-to-Text API に入力する情報を定義します.


In [0]:
from base64 import b64encode
from json import dumps

audio_filename = "ja-sample.flac" #@param {type:"string"}

with open(audio_filename, 'rb') as audio_file:
  content = b64encode(audio_file.read()).decode('utf-8')

my_audio = {
  'content': content
}

languageCode = 'ja-JP' #@param ["en-US", "ja-JP"]
model = 'default' #@param ["command_and_search", "phone_call", "video", "default"]

[RecognitionConfig](https://cloud.google.com/speech-to-text/docs/reference/rest/v1p1beta1/RecognitionConfig) を定義します。

In [0]:
my_recognition_config = {
  'encoding': 'FLAC',
  'sampleRateHertz': 16000,
  'languageCode': languageCode,
  'model': model
}

[recognize](https://cloud.google.com/speech-to-text/docs/reference/rest/v1p1beta1/speech/recognize) method のリクエストメッセージの body を定義します。

In [0]:
my_request_body={
  'audio': my_audio,
  'config': my_recognition_config,
}

[recognize](https://cloud.google.com/speech-to-text/docs/reference/rest/v1p1beta1/speech/recognize) method を実行します。

In [0]:
response = speech_service.speech().recognize(body=my_request_body).execute()

[recognize](https://cloud.google.com/speech-to-text/docs/reference/rest/v1p1beta1/speech/recognize) method のレスポンスを確認します。

In [12]:
response

{'results': [{'alternatives': [{'confidence': 0.9519596,
     'transcript': '明日の会議の時間教えて'}],
   'languageCode': 'ja-jp'}]}

In [13]:
for r in response["results"]:
    print('認識結果: ', r['alternatives'][0]['transcript'])
    print('信頼度: ', r['alternatives'][0]['confidence'])

認識結果:  明日の会議の時間教えて
信頼度:  0.9519596


### 単語のタイムスタンプの取得

[RecognitionConfig](https://cloud.google.com/speech-to-text/docs/reference/rest/v1p1beta1/RecognitionConfig) に ```enableWordTimeOffsets``` の設定を追加します。

In [0]:
my_recognition_config = {
  'encoding': 'FLAC',
  'sampleRateHertz': 16000,
  'languageCode': languageCode,
  'model': model,
  'enableWordTimeOffsets': True
}

my_request_body={
  'audio': my_audio,
  'config': my_recognition_config,
}

[recognize](https://cloud.google.com/speech-to-text/docs/reference/rest/v1p1beta1/speech/recognize) method を実行します。

In [0]:
response = speech_service.speech().recognize(body=my_request_body).execute()

[recognize](https://cloud.google.com/speech-to-text/docs/reference/rest/v1p1beta1/speech/recognize) method のレスポンスを確認します。

In [16]:
response

{'results': [{'alternatives': [{'confidence': 0.9519596,
     'transcript': '明日の会議の時間教えて',
     'words': [{'endTime': '1.900s',
       'startTime': '0s',
       'word': '明日|アシタ,アス,ミョーニチ'},
      {'endTime': '2s', 'startTime': '1.900s', 'word': 'の|ノ'},
      {'endTime': '2.300s', 'startTime': '2s', 'word': '会議|カイギ'},
      {'endTime': '2.600s', 'startTime': '2.300s', 'word': 'の|ノ'},
      {'endTime': '2.800s', 'startTime': '2.600s', 'word': '時間|ジカン'},
      {'endTime': '3.400s', 'startTime': '2.800s', 'word': '教え|オシエ'},
      {'endTime': '3.500s', 'startTime': '3.400s', 'word': 'て|テ'}]}],
   'languageCode': 'ja-jp'}]}

In [17]:
for r in response["results"]:
    print('認識結果: ', r['alternatives'][0]['transcript'])
    print('信頼度: ', r['alternatives'][0]['confidence'], "\n")

for r in response["results"][0]['alternatives'][0]["words"]:
    print("word: ", r["word"])
    print("startTime: ", r["startTime"])
    print("endTime: ", r["endTime"], "\n")

認識結果:  明日の会議の時間教えて
信頼度:  0.9519596 

word:  明日|アシタ,アス,ミョーニチ
startTime:  0s
endTime:  1.900s 

word:  の|ノ
startTime:  1.900s
endTime:  2s 

word:  会議|カイギ
startTime:  2s
endTime:  2.300s 

word:  の|ノ
startTime:  2.300s
endTime:  2.600s 

word:  時間|ジカン
startTime:  2.600s
endTime:  2.800s 

word:  教え|オシエ
startTime:  2.800s
endTime:  3.400s 

word:  て|テ
startTime:  3.400s
endTime:  3.500s 



### 演習問題

** 1. [こちら](https://cloud.google.com/speech-to-text/docs/word-confidence)を参考にして、単語レベルの信頼度を見てみましょう**


## Cloud Text-to-Speech を使ってみよう !
**Cloud Text-to-Speech** を使うと、自然な会話音声を合成できます。用意されている声は 30 種類。数多くの言語と方言に対応します。

API Discovery Service を利用して **Cloud Text-to-Speech API** を発見します。 Cloud Text-to-Speech の REST API 仕様は [こちら](https://cloud.google.com/text-to-speech/docs/reference/rest/) に解説されています。

In [0]:
import textwrap
from googleapiclient.discovery import build
service = build('texttospeech', 'v1beta1', developerKey=APIKEY)

### サポートされているすべての音声の一覧表示

テキスト読み上げ合成のために Cloud Text-to-Speech API で使用できる音声を一覧表示します。なお、 ```languageCode``` は[こちら](https://cloud.google.com/text-to-speech/docs/voices)を参考にしてください。

In [0]:
response = service.voices().list(
  languageCode="ja_JP",
).execute()

In [22]:
for voice in response['voices']:
  print(voice)

{'languageCodes': ['ja-JP'], 'name': 'ja-JP-Wavenet-A', 'ssmlGender': 'FEMALE', 'naturalSampleRateHertz': 24000}
{'languageCodes': ['ja-JP'], 'name': 'ja-JP-Standard-A', 'ssmlGender': 'FEMALE', 'naturalSampleRateHertz': 22050}


### テキストから音声を合成する

[text.synthesize](https://cloud.google.com/text-to-speech/docs/reference/rest/v1beta1/text/synthesize) メソッドを使用すると、単語や文を自然な人間の音声の base64 でエンコードされた音声データに変換できます。このメソッドは、入力を生のテキストまたは[音声合成マークアップ言語（SSML）](https://cloud.google.com/text-to-speech/docs/ssml)として受け入れます。

In [0]:
source_language = "ja_JP" #@param {type: "string"}
source_sentence = "Google Cloud Text-to-Speech \u3092\u4F7F\u3046\u3068\u3001\u81EA\u7136\u306A\u4F1A\u8A71\u97F3\u58F0\u3092\u5408\u6210\u3067\u304D\u307E\u3059\u3002" #@param {type:"string"}
audio_encoding = 'OGG_OPUS' #@param ['OGG_OPUS', 'LINEAR16', 'MP3']
voice_gender = 'FEMALE' #@param ['FEMALE', 'MALE', 'NEUTRAL', 'SSML_VOICE_GENDER_UNSPECIFIED']
textwrap.wrap(source_sentence)
voice_name = 'ja-JP-Wavenet-A' #@param {type: "string"}

In [0]:
response = service.text().synthesize(
  body={
    'input': {
      'text': source_sentence,
    },
    'voice': {
      'languageCode': source_language,
      'ssmlGender': voice_gender,
      'name': voice_name,
    },
    'audioConfig': {
      'audioEncoding': audio_encoding,
    },
  }
).execute()

合成した音声を確認しましょう

In [25]:
import base64
from IPython.display import Audio

Audio(base64.b64decode(response['audioContent']))

### 演習問題

** 1. 日本語のテキストを Standard モデルで音声合成してみましょう。**

**2. 英語のテキストを様々なモデルで音声合成してみましょう。**